## Direitos Autorais Modelo ChatGPT em R:
Modelo apresentado na aula 27 do curso de estatística e Machine Learning do IME-USP.

Autoria: Prof. Alexandre Galvão Patriota

## 🔹 Parâmetros de Entrada do Modelo GPT

A função de inicialização do modelo recebe os seguintes argumentos:

| **Parâmetro** | **Significado** | **Interpretação** |
|----------------|-----------------|-------------------------------|
| **block_size** | Tamanho da janela de contexto (número de tokens consecutivos considerados no bloco attention). | Define o comprimento da janela de condicionamento — quantos tokens anteriores o modelo utiliza para prever o próximo. |
| **n_embd** | Dimensão do vetor de embedding. | Tamanho do espaço latente contínuo onde cada token é representado por um vetor real. |
| **N_Layers** | Número de camadas do Transformer (profundidade da rede). | Quantas vezes o bloco “Atenção + Feed-Forward + Normalização” é repetido ao longo da rede. |
| **nvoc** | Tamanho do vocabulário. | Número de categorias possíveis no modelo multinomial. Cada predição escolhe uma entre `nvoc` opções (No método Greedy). |
| **head** | Número de *heads* de atenção. | Quantas projeções paralelas de atenção são calculadas — cada *head* modela um tipo distinto de dependência contextual. |
| **p0** | Taxa de *dropout* (padrão 0.1). | Probabilidade de zerar aleatoriamente algumas ativações durante o treino, reduzindo o *overfitting*. |

---

In [5]:
library(torch)

GPT <- torch::nn_module(
  initialize = function(block_size, n_embd, N_Layers, nvoc, Head, p0 = 0.1) {

    self$N   <- N_Layers
    self$wpe <- torch::nn_embedding(block_size, n_embd)
    self$wte <- torch::nn_embedding(nvoc, n_embd, padding_idx = 1)

    self$MM  <- torch::nn_module_list(lapply(
      1:N_Layers,
      function(x) torch::nn_multihead_attention(n_embd, Head, dropout = p0, batch_first = TRUE)
    ))

    self$scale1 <- torch::nn_module_list(lapply(
      1:N_Layers,
      function(x) torch::nn_layer_norm(n_embd)
    ))

    self$scale2 <- torch::nn_module_list(lapply(
      1:N_Layers,
      function(x) torch::nn_layer_norm(n_embd)
    ))

    self$scale3 <- torch::nn_layer_norm(n_embd, elementwise_affine = TRUE)

    self$FFN <- torch::nn_module_list(lapply(
      1:N_Layers,
      function(x) {
        torch::nn_sequential(
          torch::nn_linear(n_embd, 4 * n_embd),
          torch::nn_gelu(),
          torch::nn_linear(4 * n_embd, n_embd),
          torch::nn_dropout(p0)
        )
      }
    ))

    # cabeça linear de saída (mantive seu nome ln_f)
    self$ln_f  <- torch::nn_linear(n_embd, nvoc, bias = FALSE)
    self$drop0 <- torch::nn_dropout(p = p0)
  },

  forward = function(x, return_intermediates = FALSE) {
    # x: (B, T)
    B <- x$size(1)
    T <- x$size(2)

    # posições 1..T (long)
    x1 <- torch::torch_arange(1, T,
      dtype = torch::torch_long(),
      device = x$device
    )

    # máscara causal (proíbe olhar para o futuro)
    wei <- torch::torch_triu(torch::torch_ones(T, T, device = x$device), diagonal = 1)$to(
      dtype = torch::torch_bool()
    )

    # embeddings
    output <- self$wte(x) + self$wpe(x1)$unsqueeze(1)  # (B, T, E)
    output <- self$drop0(output)

    # (opcional) inspeção rápida
    # cat("wei shape:", as.character(wei$size()), "\n"); print(wei$to(dtype = torch_int()))
    # cat("x1 shape:", as.character(x1$size()), "\n"); print(x1)

    for (j in 1:self$N) {
      # pré-norm + atenção multihead
      QKV <- self$scale1[[j]](output)  # (B, T, E) pois batch_first = TRUE
      attn_out <- self$MM[[j]](
        query = QKV, key = QKV, value = QKV,
        attn_mask = wei, need_weights = FALSE
      )[[1]]
      output <- output + attn_out

      # feed-forward com pré-norm
      output <- output + self$FFN[[j]](self$scale2[[j]](output))
    }

    # norm final + cabeça linear → logits (B, T, nvoc)
    output <- self$scale3(output)
    logits <- self$ln_f(output)

    if (return_intermediates) {
      return(list(
        x1    = x1$to("cpu"),
        wei   = wei$to(dtype = torch_int())$to("cpu"),
        out   = output$to("cpu"),
        logits = logits$to("cpu")
      ))
    }
    logits
  }
)

In [10]:
# Criar uma instância do modelo
model <- GPT(
  block_size = 4, #8
  n_embd = 8, #16
  N_Layers = 2, #2
  nvoc = 4, #20
  Head = 2 #2
)

In [11]:
# Criar uma entrada fictícia (batch de tokens)
x <- torch::torch_tensor(matrix(c(1,2,3,4,5,6,7,8), nrow = 1))

# Executar o modelo
res <- model(x, return_intermediates = TRUE)

# Visualizar variáveis intermediárias
res$x1
res$wei

ERROR: Error in (function (weight, indices, padding_idx, scale_grad_by_freq, : Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got CPUFloatType instead (while checking arguments for embedding)
Exception raised from checkScalarTypes at /Users/runner/work/libtorch-mac-m1/libtorch-mac-m1/pytorch/aten/src/ATen/TensorUtils.cpp:203 (most recent call first):
frame #0: c10::Error::Error(c10::SourceLocation, std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char>>) + 52 (0x10456c55c in libc10.dylib)
frame #1: c10::detail::torchCheckFail(char const*, char const*, unsigned int, std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char>> const&) + 140 (0x1045691ac in libc10.dylib)
frame #2: at::checkScalarTypes(char const*, at::TensorArg const&, c10::ArrayRef<c10::ScalarType>) + 480 (0x3000c36a0 in libtorch_cpu.dylib)
frame #3: at::native::embedding_symint(at::Tensor const&, at::Tensor const&, 